In [27]:
import yaml
import pandas as pd
import numpy as np

In [28]:
# Paramètres
with open("config.yml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

INPUT_FICHIER_WIKI_TEXTE = config["camins"]["texte"]["fichier_wiki_texte"]
OUTPUT_FICHER_WIKI_ETIQUETAS_AUTOMATIC = config["camins"]["etiquetas"]["fichier_wiki_etiquetas_automatic"]

MIN_LEN = int(config["parametres"]["longor_min"])
MAX_LEN = config["parametres"]["longor_max"]
if MAX_LEN == "inf":
    MAX_LEN = np.inf

DIALECTES = config["dialectes"]

with open("mots_tipics.yml", "r", encoding="utf-8") as file:
    MOTS_TIPICS: dict[str, set] = {dialecte: set(mots) for dialecte, mots in yaml.safe_load(file).items()}
print(MOTS_TIPICS)

{'lengadocian': set(), 'gascon': {'ei', 'puish', 'enter', 'eth', 'castèth', 'har', 'entà', 'ahar', 'neishut', 'hèsta', 'damb', 'shens', 'peu', 'suu', 'dab', 'hilh', 'medish'}, 'auvernhat': {"z'a", 'z-ai', 'z-èra', "z'es", 'z-a', "z'ai", 'embe', 'atí', 'ilhs', 'ilh', "z'èra", 'z-es', 'puesche'}, 'provençal': {'toti', 'sei', 'dei', 'elei', 'avans', 'pòu', 'ambé', 'ansin'}, 'francés': {'cette', 'mes', 'être', 'tes', 'il', 'à', 'parce que', 'ce', 'même', 'bien', 'ces', 'avoir', 'une', 'aux', 'étaient', 'dont', 'le', 'pour', 'je', 'dans', 'était', 'ici', 'des', 'toi', 'elle', 'les', 'moi', 'lui'}}


In [29]:
# Cargar donadas
df_tot = pd.read_csv(INPUT_FICHIER_WIKI_TEXTE, sep=",", dtype=str).set_index("Article")

In [30]:
# Filtrar tèxtes per longor
longor_texte = df_tot["Texte"].apply(len)
df_sub = df_tot[(longor_texte > MIN_LEN) & (longor_texte < MAX_LEN)].copy()

In [31]:
def contar_mots_tipics(texte: str, mots_tipics: set) -> int:
    """
    Comptar mots tipics dins un tèxte.
    """
    return len(set(texte.lower().split()) & mots_tipics)


def custom_idxmax(row: pd.Series):
    """
    Trobar lo dialecte amb mai de mots tipics.
    """
    sorted_values = row.sort_values(ascending=False)
    if sorted_values.iloc[0] == sorted_values.iloc[1]:
        return np.nan
    else:
        return sorted_values.idxmax()


# Calcul dels mots tipics per cada dialecte
for dialecte, mots_tipics in MOTS_TIPICS.items():
    df_sub[dialecte] = df_sub["Texte"].apply(contar_mots_tipics, mots_tipics=mots_tipics)
df_sub = df_sub.loc[df_sub[DIALECTES].sum(axis=1) > 0].copy()
df_sub["Dialecte"] = df_sub[DIALECTES].apply(custom_idxmax, axis=1)

In [32]:
# Filtrar tèxtes amb mots tipics etiquetats
df_etiquetat_mots_tipics = df_sub[
    (df_sub[DIALECTES] >= 1).any(axis=1) & df_sub["Dialecte"].notna() & (df_sub["francés"] == 0)
]
df_etiquetat_mots_tipics

,Texte,lengadocian,gascon,auvernhat,provençal,francés,Dialecte
Article,,,,,,,
Vladimir_Arnold,vladimir igorevich arnold (12 de junh de 1937 ...,0,0,0,1,0,provençal
Cuelha,cuelha (queuille en francés) z-es una comuna o...,0,0,1,0,0,auvernhat
Auriòu_(Provença),"auriòu (auruou en provençau, auriol en francés...",0,0,0,1,0,provençal
Pierre-Simon_de_Laplace,"pierre-simon de laplace (23 de març de 1749, b...",0,0,0,1,0,provençal
Mirza_Ghalib,mirza asadullah khan ghalib (27 de decembre de...,0,0,0,1,0,provençal
...,...,...,...,...,...,...,...
Aurelian_(emperaire_roman),"aurelian , en latin: lucius domitius aurelianu...",0,0,0,2,0,provençal
Miranda_(Gers),miranda (mirande en francés) qu'ei ua comuna o...,0,2,0,0,0,gascon
Lekunberri_(Baisha_Navarra),"lekunberri (basco), lecumberry (francés), qu'e...",0,1,0,0,0,gascon


In [33]:
# Salvar las etiquetas automaticas
df_etiquetat_mots_tipics[["Dialecte"]].to_csv(OUTPUT_FICHER_WIKI_ETIQUETAS_AUTOMATIC)